In [1]:
#  신경망 출력값과 정답 비교
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))
import custom

import pickle
import numpy as np
import onnxruntime


In [2]:
# 테스트 할 데이터 불러오기

with open("data/sound_data_train.pkl", mode = "rb") as f:
    sound_data = pickle.load(f)
with open("data/text_data_train.pkl", mode = "rb") as f:
    text_data = pickle.load(f)

# 음절 사전
with open("sound_dict.pkl", mode = "rb") as f:
    sound_dict = pickle.load(f) #음절-라벨 사전
num_dict = {v : k for k, v in sound_dict.items()} #라벨-음절 사전

In [3]:
# encoder, decoder 불러오기

encoder = onnxruntime.InferenceSession("encoder.onnx", providers=["CPUExecutionProvider"])
decoder = onnxruntime.InferenceSession("decoder.onnx", providers=["CPUExecutionProvider"])

encoder_inputs = encoder.get_inputs()
decoder_inputs = decoder.get_inputs()

In [4]:
sos_idx = sound_dict['<sos>']
eos_idx = sound_dict['<eos>']

start_idx = 2001
end_idx = 2020

outputs_list = []
for i in range(start_idx, end_idx+1) :
    total_output = []
    
    x = sound_data[i:i+1] #소리 데이터 하나
    # print(x.shape)
    
    # encoder 계산
    x = x.astype(np.float32)
    inputs = {encoder_inputs[0].name : x}
    outs = encoder.run(None, inputs)
    encoder_y = outs[0]
    h = outs[1]
    c = outs[2]
    # decoder 계산
    x = np.full((1,1), sos_idx).astype(np.int64) #처음 x가 필요, <SOS부터 시작>
    while True:
        inputs = {decoder_inputs[0].name : x, decoder_inputs[1].name : h, decoder_inputs[2].name : c, decoder_inputs[3].name : encoder_y}
        outs = decoder.run(None, inputs)
        decoder_y = outs[0]
        h = outs[1]
        c = outs[2]
        x = np.argmax(decoder_y, axis = -1)
        total_output.append(num_dict[x.item()])
        if x.item() == eos_idx :
            break
    outputs_list.append("".join(total_output))

for a in outputs_list :
    print(a)

캐삭빵<eos>
쿼드라킬<eos>
크리<eos>
크카<eos>
탑신병자<eos>
탱커<eos>
템빨<eos>
템전<eos>
트리플킬<eos>
팀킬<eos>
파덜어택<eos>
펜타킬<eos>
풀방<eos>
피방<eos>
피흡<eos>
필수템<eos>
팀킬<eos>
하스데<eos>
한타<eos>
헤드샷<eos>


In [5]:
# print(text_data[start_idx:end_idx+1])

text_list = []
for i in range(start_idx, end_idx+1) :
    text = custom.word_vectorize(text_data[i], num_dict)
    text_list.append("".join(text))

for a in text_list :
    print(a)

캐삭빵<eos><pad><pad>
쿼드라킬<eos><pad>
크리<eos><pad><pad><pad>
크카<eos><pad><pad><pad>
탑신병자<eos><pad>
탱커<eos><pad><pad><pad>
템빨<eos><pad><pad><pad>
템전<eos><pad><pad><pad>
트리플킬<eos><pad>
팀킬<eos><pad><pad><pad>
파덜어택<eos><pad>
펜타킬<eos><pad><pad>
풀방<eos><pad><pad><pad>
피방<eos><pad><pad><pad>
피흡<eos><pad><pad><pad>
필수템<eos><pad><pad>
필킬<eos><pad><pad><pad>
하스데<eos><pad><pad>
한타<eos><pad><pad><pad>
헤드샷<eos><pad><pad>
